# M4 Agentic AI - 为研究工作流添加组件级评估

## 1. 引言

在上一份评分实验（M3）中，你构建了一个会使用工具的研究型代理，执行了由三步组成的工作流：

1. 在网上搜索信息。  
2. 对其输出进行反思。  
3. 发布一份清晰的 HTML 报告。  

在本次无评分实验中，你将专注于评估该工作流中的**一个组件**：即*研究步骤*。  

不同于生成和润色文章，这里你将设计一个**组件级评估**，用于检查研究步骤返回的来源质量。  

该评估会将代理检索到的 URL 与一份**预定义的优选域名列表**进行对比（例如：`arxiv.org`、`nature.com`、`nasa.gov`）。  

这使你能够用**客观的、按示例的真实标准**来量化系统是否从可信来源获取信息。


### 1.1. 实验概览

在课程视频中，Andrew 展示了一个网页搜索结果**质量较差**的案例，使得检索到的信息难以被信任。  
基于该示例，在本实验中你将通过与**预定义的优选域名列表**对比来评估来源的可靠性。

在此次评估中，我们将聚焦主题*“黑洞科学的最新进展”*，这是课程中强调的示例之一。  
我们的想法是验证网页搜索工具是否返回了来自优选域名的来源，并量化优选结果与总结果的比例。

该评估将以一个函数实现，执行**客观的、按示例的检查**：

* 解析 Tavily 的输出（我们的网页搜索工具）。  
* 识别哪些 URL 属于**优选域名**列表。  
* 计算优选来源占已检索来源的比例。  
* 返回一个布尔标志（**通过/未通过**），以及可直接嵌入报告的 Markdown 格式摘要。  

<img src="M4-UGL-1.png" width="70%">  


### 1.2. 🎯 学习目标

你将学到：

* 编写一个函数，检查网页搜索 API 的结果是否来自**优选来源**。  
* 创建一个评估，用于验证你的来源是否来自**优选域名**。  
* 将一个**组件级评估**加入网页搜索函数。  


## 2. 环境设置：导入库并加载环境

与之前的实验一样，你首先导入所需的库并初始化运行环境。

In [ ]:
# =========================
# 导入
# =========================

# --- 标准库 
from datetime import datetime
import json
import re

# --- 第三方 ---
from aisuite import Client

# --- 本地 / 项目 ---
import research_tools
import utils

client = Client()

## 3. 研究步骤 – `find_references`

在评分实验中，你实现的函数在同一步中同时完成了**网页搜索与草稿报告撰写**。

这里我们将网页搜索拆分为一个名为 `find_references` 的独立函数。这样你可以在不考虑撰写和反思步骤的情况下，单独评估搜索结果；本实验只关注网页搜索步骤的输出。

与评分实验的实现相比，这里有两个关键差异：

* 新函数使用 **AISuite**，它会为你自动管理工具调用（而不是用 OpenAI SDK 手写工具调用代码）。  
* 该函数还会告知 LLM **当前日期**，这有助于提升时间敏感查询的相关性。  

`find_references` 的角色是从 **Arxiv**、**Tavily** 和 **Wikipedia** 等工具中**收集外部信息**。  
由于这些结果的质量将直接影响评分实验的输出，因此这是可以应用**评估方法**的阶段——例如，检查返回的 URL 是否来自你的**优选域名**列表。  

In [ ]:
def find_references(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """使用外部工具（arxiv、tavily、wikipedia）执行研究任务。"""

    prompt = f"""
    你是一个研究函数，可以访问：
    - arxiv_tool：学术论文
    - tavily_tool：通用网页搜索（需要时返回 JSON）
    - wikipedia_tool：百科式摘要

    任务：
    {task}

    今天是 {datetime.now().strftime('%Y-%m-%d')}。
    """.strip()

    messages = [{"role": "user", "content": prompt}]
    tools = [
        research_tools.arxiv_search_tool,
        research_tools.tavily_search_tool,
        research_tools.wikipedia_search_tool,
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_turns=5,
        )
        content = response.choices[0].message.content
        return (content, messages) if return_messages else content
    except Exception as e:
        return f"[模型错误: {e}]"

运行下面的单元以测试研究函数。  
该任务将检索两篇关于黑洞科学最新进展的论文，并显示结果。

In [ ]:
research_task = "查找 2 篇关于黑洞科学最新进展的近期论文"
research_result = find_references(research_task)

utils.print_html(
    research_result,
    title="研究函数输出"
)

## 4. 评估步骤 – 优选域名

并非所有网页搜索检索到的来源都同样可靠。  
在本实验中，我们聚焦于**上一份评分实验中的一个步骤**——`find_references` 研究步骤——并展示如何设计一个**组件级评估**，用于检查返回的域名是否属于预定义的**优选域名**列表。  

这是一个**具有明确按示例真实标准**的**客观评估**案例。  
回顾课程中的评估两个维度：在这些维度上，我们位于**左上象限**——按示例、具有明确真实标准的客观评估。

<img src='M4-UGL-1-Evaluations.png' width='80%'>


### 为什么需要组件级评估？

正如 Andrew 在讲座中所说：  

- 如果问题出在网页搜索（通常是评分实验工作流中的**第一步**），每次重跑*整条*流水线（搜索 → 草稿 → 反思）会非常**昂贵**且噪声多。  
- 网页搜索质量的小幅提升，可能会被后续组件引入的随机性所掩盖。  
- 通过只评估网页搜索，你可以获得该组件是否改善的**更清晰信号**。  

当多支团队分别负责系统的不同部分时，组件级评估也很高效：每个团队都可以用清晰的指标优化自己的组件，而无需运行或等待完整的端到端测试。  

### 我们如何评估？

这里的评估是**客观的**，因此可以用代码进行评估。它具有示例特定的真实标准——本次黑洞主题的优选来源列表。要构建该评估，你将：

1. 提取 Tavily 返回的 URL。  
2. 将其与预定义的**优选域名**列表对比（例如 `arxiv.org`、`nature.com`、`nasa.gov`）。  
3. 计算**优选结果与总结果的比例**。  
4. 返回一个**通过/未通过**标志，并附带 Markdown 格式摘要。  

这提供了一个可复现、低成本的度量，告诉我们研究组件——也就是评分实验中的这一步——是否从可信来源获取信息。



In [ ]:
# Tavily 结果的首选域名列表
TOP_DOMAINS = {
    # 通用参考 / 机构 / 出版方
    "wikipedia.org", "nature.com", "science.org", "sciencemag.org", "cell.com",
    "mit.edu", "stanford.edu", "harvard.edu", "nasa.gov", "noaa.gov", "europa.eu",

    # 计算机科学 / 人工智能领域会议与索引
    "arxiv.org", "acm.org", "ieee.org", "neurips.cc", "icml.cc", "openreview.net",

    # 其他权威出版源
    "elifesciences.org", "pnas.org", "jmlr.org", "springer.com", "sciencedirect.com",

    # 额外域名（针对特定情况）
    "pbs.org", "nova.edu", "nvcc.edu", "cccco.edu",

    # 知名编程网站
    "codecademy.com", "datacamp.com"
}

def evaluate_tavily_results(TOP_DOMAINS, raw: str, min_ratio=0.4):
    """
    评估纯文本研究结果是否主要来自首选域名。

    Args:
        TOP_DOMAINS (set[str]): 首选域名集合（例如 'arxiv.org', 'nature.com'）。
        raw (str): 包含 URL 的纯文本或 Markdown。
        min_ratio (float): 通过评估所需的最小首选比例（例如 0.4 = 40%）。

    Returns:
        tuple[bool, str]: (flag, markdown_report)
            flag -> True 表示通过，False 表示未通过
            markdown_report -> Markdown 格式的评估摘要
    """

    # 从文本中提取 URL
    url_pattern = re.compile(r'https?://[^\s\]\)>\}]+', flags=re.IGNORECASE)
    urls = url_pattern.findall(raw)

    if not urls:
        return False, """### Evaluation — Tavily Preferred Domains
未检测到任何 URL。
请在研究结果中包含链接。
"""

    # 统计首选域名与总数
    total = len(urls)
    preferred_count = 0
    details = []

    for url in urls:
        domain = url.split("/")[2]
        preferred = any(td in domain for td in TOP_DOMAINS)
        if preferred:
            preferred_count += 1
        details.append(f"- {url} → {'✅ 首选域名' if preferred else '❌ 非首选域名'}")

    ratio = preferred_count / total if total > 0 else 0.0
    flag = ratio >= min_ratio

    # 生成 Markdown 格式的评估报告
    report = f"""
### 评估结果 — Tavily 首选域名
- 总链接数: {total}
- 首选链接数: {preferred_count}
- 比例: {ratio:.2%}
- 阈值: {min_ratio:.0%}
- 状态: {"✅ 通过" if flag else "❌ 未通过"}

**详细结果：**
{chr(10).join(details)}
"""
    return flag, report


<div style="border:1px solid #93c5fd; border-left:6px solid #3b82f6; background:#dbeafe; border-radius:6px; padding:12px 14px; color:#1e3a8a; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">  
<strong>🔎 为什么这是客观评估：</strong><br><br>  
从 Tavily 检索到的每个 URL 都会与预定义的<em>优选域名</em>（<code>TOP_DOMAINS</code>）进行对比：<br>  
• 若域名匹配 → ✅ 优选<br>  
• 否则 → ❌ 非优选<br><br>  
是否通过取决于优选来源的比例是否超过设定阈值，因此会产生清晰的通过/未通过信号。  
由于每个示例的真实标准（优选 vs 非优选）都被明确地定义，评估既<strong>客观</strong>又<strong>可复现</strong>。  
</div>


运行该单元以显示优选域名示例、研究结果以及评估摘要（通过/未通过及详细信息）。

In [ ]:
utils.print_html(json.dumps(list(TOP_DOMAINS)[:4], indent=2), title="优选域名示例")

utils.print_html("<h3>研究结果</h3>" + research_result, title="研究结果")

flag, report = evaluate_tavily_results(TOP_DOMAINS, research_result)
utils.print_html("<pre>" + report + "</pre>", title="<h3>评估摘要</h3>")

## 动手试试！

现在轮到你了。  
在本节中，你可以直接试验**研究步骤**及其**评估**：  

* **主题**：选择一个不同的研究主题。  
* **优选域名**：编辑或扩展 `TOP_DOMAINS` 列表。  
* **评估比例**：调整 `min_ratio`（例如 0.4 = 至少 40% 为优选来源）。  

修改后重新运行下面的单元，观察评估如何变化。


In [ ]:
# === 5.1. 自主尝试：主题、比例与优选域名 ===
# 在运行单元前编辑这些参数

topic = "黑洞科学的最新进展"   # <- 在此修改主题
min_ratio = 0.4                   # <- 修改阈值（0.0–1.0）
run_reflection = True             # <- 设为 False 跳过步骤 4

# 简短的优选域名列表（按需编辑或扩展）
TOP_DOMAINS = {
    "wikipedia.org", "nature.com", "science.org", "arxiv.org",
    "nasa.gov", "mit.edu", "stanford.edu", "harvard.edu"
}

# 显示优选域名示例
import json
utils.print_html(
    json.dumps(sorted(list(TOP_DOMAINS)), indent=2),
    title="<h3>优选域名示例</h3>"
)

# 1) 研究
research_task = f"查找 {topic} 的 2–3 篇关键论文与可靠综述。"
research_output = find_references(research_task)
utils.print_html(research_output, title=f"<h3>{topic} 的研究结果</h3>")

# 2) 评估来源（优选域名比例）
flag, eval_md = evaluate_tavily_results(TOP_DOMAINS, research_output, min_ratio=min_ratio)
utils.print_html("<pre>" + eval_md + "</pre>", title="<h3>评估摘要</h3>")

## 5. 要点总结

* 你刚刚学习了如何评估**一个组件**的性能：即 `find_references` 研究步骤。  
* 你的组件级评估检查了检索到的 URL 是否在预定义的**优选域名**列表中。  
* 这是一个具有明确**按示例真实标准**的**客观评估**示例。  
* 为构建评估集，你可以设计约 10 个涵盖不同主题（天文学、机器人、金融等）的提示，并为每个主题定义优选域名。  
* 与优选域名列表匹配的来源百分比提供了有用的**指标**，可指导改进，例如调整提示或工具参数。  
* 这种方法比评估包含反思与重写的整篇文章**更简单也更便宜**，因为你仅聚焦于网页搜索组件。  

<div style="border:1px solid #22c55e; border-left:6px solid #16a34a; background:#dcfce7; border-radius:6px; padding:14px 16px; color:#064e3b; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

🎉 **恭喜！**  

你设计了一个**组件级评估**，使你的研究代理更加可靠。  
通过直接检查来源质量，你引入了一个**客观、可复现、且高性价比**的保障。  

这与 Andrew 的讲座中强调的理念一致：*组件级评估*让你无需评估整条流水线就能测试 AI 系统的单个部分。  

</div>


